# Exploring Hacker News Post: Which way to go - Ask or Show Hacker News?
This project seeks to discover the category of post that receives more comments on Hacker News - Ask Hacker News (Ask HN) or Show Hacker News (Show HN) posts?

Hacker News is a popular site where technology related stories (or 'posts') are voted and commented upon. The two types of posts we'll explore begin with either Ask HN or Show HN. Users submit Ask HN posts to ask the Hacker News community a specific question, such as "What is the best online course you've ever taken?" Likewise, users submit Show HN posts to show the Hacker News community a project, product, or just generally something interesting.

We'll specifically compare these two types of posts to determine the following:

Do Ask HN or Show HN receive more comments on average? Do posts created at a certain time receive more comments on average? It should be noted that the data set we're working with was reduced from almost 300,000 rows to approximately 20,000 rows by removing all submissions that did not receive any comments, and then randomly sampling from the remaining submissions.

The data for this project and the column descriptions can be found [here](https://www.kaggle.com/hacker-news/hacker-news-posts)

The total user posts is approximately 300,000. For this project, we have removed posts/user submissions with no comments and are left with a randomly sliced out 20,000 submissions.

# Introduction
First, we'll read in the data and remove the headers.

In [1]:
# Read in the data.
import csv
f = open('hacker_news.csv')
hn = list(csv.reader(f))
hn[:5]

[['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at'],
 ['12224879',
  'Interactive Dynamic Video',
  'http://www.interactivedynamicvideo.com/',
  '386',
  '52',
  'ne0phyte',
  '8/4/2016 11:52'],
 ['10975351',
  'How to Use Open Source and Shut the Fuck Up at the Same Time',
  'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/',
  '39',
  '10',
  'josep2',
  '1/26/2016 19:30'],
 ['11964716',
  "Florida DJs May Face Felony for April Fools' Water Joke",
  'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/',
  '2',
  '1',
  'vezycash',
  '6/23/2016 22:20'],
 ['11919867',
  'Technology ventures: From Idea to Enterprise',
  'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429',
  '3',
  '1',
  'hswarna',
  '6/17/2016 0:01']]

# Removing Headers from a List of Lists

In [2]:
# Remove the headers.
headers = hn[0]
hn = hn[1:]
print(headers)
print(hn[:5])

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']
[['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52'], ['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30'], ['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20'], ['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01'], ['10301696', 'Note by Note: The Making of Steinway L1037 (2007)', 'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0', '8', '2', 'walterbell', '9/30/2015 4:12']]


We can see above that the data set contains the title of the posts, the number of comments for each post, and the date the post was created. Let's start by exploring the number of comments for each type of post.

# Extracting Ask HN and Show HN Posts¶
First, we'll identify posts that begin with either Ask HN or Show HN and separate the data for those two types of posts into different lists. Separating the data makes it easier to analyze in the following steps.

In [3]:
# Identify posts that begin with either `Ask HN` or `Show HN` and separate the data into different lists.
ask_posts = []
show_posts =[]
other_posts = []

for post in hn:
    title = post[1]
    if title.lower().startswith("ask hn"):
        ask_posts.append(post)
    elif title.lower().startswith("show hn"):
        show_posts.append(post)
    else:
        other_posts.append(post)
        
print(len(ask_posts))
print(len(show_posts))
print(len(other_posts))

1744
1162
17194


# Calculating the Average Number of Comments for Ask HN and Show HN Posts¶
Now that we separated ask posts and show posts into different lists, we'll calculate the average number of comments each type of post receives.

In [4]:
# Calculate the average number of comments `Ask HN` posts receive.
total_ask_comments = 0

for post in ask_posts:
    total_ask_comments += int(post[4])
    
avg_ask_comments = total_ask_comments / len(ask_posts)
print(avg_ask_comments)
    

14.038417431192661


In [5]:

total_show_comments = 0

for post in show_posts:
    total_show_comments += int(post[4])
    
avg_show_comments = total_show_comments / len(show_posts)
print(avg_show_comments)

10.31669535283993


On average, ask posts in our sample receive approximately 14 comments, whereas show posts receive approximately 10. Since ask posts are more likely to receive comments, we'll focus our remaining analysis just on these posts

# Finding the Amount of Ask Posts and Comments by Hour Created
Next, we'll determine if we can maximize the amount of comments an ask post receives by creating it at a certain time. First, we'll find the amount of ask posts created during each hour of day, along with the number of comments those posts received. Then, we'll calculate the average amount of comments ask posts created at each hour of the day receive.

In [6]:
import datetime as dt

result_list = []

for post in ask_posts:
    result_list.append(
        [post[6], int(post[4])]
    )

comments_by_hour = {}
counts_by_hour = {}
date_format = "%m/%d/%Y %H:%M"

for each_row in result_list:
    date = each_row[0]
    comment = each_row[1]
    time = dt.datetime.strptime(date, date_format).strftime("%H")
    if time in counts_by_hour:
        comments_by_hour[time] += comment
        counts_by_hour[time] += 1
    else:
        comments_by_hour[time] = comment
        counts_by_hour[time] = 1

comments_by_hour

{'09': 251,
 '13': 1253,
 '10': 793,
 '14': 1416,
 '16': 1814,
 '23': 543,
 '12': 687,
 '17': 1146,
 '15': 4477,
 '21': 1745,
 '20': 1722,
 '02': 1381,
 '18': 1439,
 '03': 421,
 '05': 464,
 '19': 1188,
 '01': 683,
 '22': 479,
 '08': 492,
 '04': 337,
 '00': 447,
 '06': 397,
 '07': 267,
 '11': 641}

# Calculating the Average Number of Comments for Ask HN Posts by Hour

In [7]:
# Calculate the average amount of comments `Ask HN` posts created at each hour of the day receive.
avg_by_hour = []

for hr in comments_by_hour:
    avg_by_hour.append([hr, comments_by_hour[hr] / counts_by_hour[hr]])

avg_by_hour

[['09', 5.5777777777777775],
 ['13', 14.741176470588234],
 ['10', 13.440677966101696],
 ['14', 13.233644859813085],
 ['16', 16.796296296296298],
 ['23', 7.985294117647059],
 ['12', 9.41095890410959],
 ['17', 11.46],
 ['15', 38.5948275862069],
 ['21', 16.009174311926607],
 ['20', 21.525],
 ['02', 23.810344827586206],
 ['18', 13.20183486238532],
 ['03', 7.796296296296297],
 ['05', 10.08695652173913],
 ['19', 10.8],
 ['01', 11.383333333333333],
 ['22', 6.746478873239437],
 ['08', 10.25],
 ['04', 7.170212765957447],
 ['00', 8.127272727272727],
 ['06', 9.022727272727273],
 ['07', 7.852941176470588],
 ['11', 11.051724137931034]]

# Sorting and Printing Values from a List of Lists

In [8]:
swap_avg_by_hour = []

for row in avg_by_hour:
    swap_avg_by_hour.append([row[1], row[0]])
    
print(swap_avg_by_hour)

sorted_swap = sorted(swap_avg_by_hour, reverse=True)

sorted_swap

[[5.5777777777777775, '09'], [14.741176470588234, '13'], [13.440677966101696, '10'], [13.233644859813085, '14'], [16.796296296296298, '16'], [7.985294117647059, '23'], [9.41095890410959, '12'], [11.46, '17'], [38.5948275862069, '15'], [16.009174311926607, '21'], [21.525, '20'], [23.810344827586206, '02'], [13.20183486238532, '18'], [7.796296296296297, '03'], [10.08695652173913, '05'], [10.8, '19'], [11.383333333333333, '01'], [6.746478873239437, '22'], [10.25, '08'], [7.170212765957447, '04'], [8.127272727272727, '00'], [9.022727272727273, '06'], [7.852941176470588, '07'], [11.051724137931034, '11']]


[[38.5948275862069, '15'],
 [23.810344827586206, '02'],
 [21.525, '20'],
 [16.796296296296298, '16'],
 [16.009174311926607, '21'],
 [14.741176470588234, '13'],
 [13.440677966101696, '10'],
 [13.233644859813085, '14'],
 [13.20183486238532, '18'],
 [11.46, '17'],
 [11.383333333333333, '01'],
 [11.051724137931034, '11'],
 [10.8, '19'],
 [10.25, '08'],
 [10.08695652173913, '05'],
 [9.41095890410959, '12'],
 [9.022727272727273, '06'],
 [8.127272727272727, '00'],
 [7.985294117647059, '23'],
 [7.852941176470588, '07'],
 [7.796296296296297, '03'],
 [7.170212765957447, '04'],
 [6.746478873239437, '22'],
 [5.5777777777777775, '09']]

In [9]:
# Sort the values and print the the 5 hours with the highest average comments.

print("Top 5 Hours for 'Ask HN' Comments")
for avg, hr in sorted_swap[:5]:
    print(
        "{}: {:.2f} average comments per post".format(
            dt.datetime.strptime(hr, "%H").strftime("%H:%M"),avg
        )
    )

Top 5 Hours for 'Ask HN' Comments
15:00: 38.59 average comments per post
02:00: 23.81 average comments per post
20:00: 21.52 average comments per post
16:00: 16.80 average comments per post
21:00: 16.01 average comments per post


The hour that receives the most comments per post on average is 15:00, with an average of 38.59 comments per post. There's about a 60% increase in the number of comments between the hours with the highest and second highest average number of comments.

According to the [data set documentation](https://www.kaggle.com/hacker-news/hacker-news-posts), the timezone used is Eastern Time in the US. So, we could also write 15:00 as 3:00 pm est and 8:00 pm WAT

# Conclusion
In this project, we analyzed ask posts and show posts to determine which type of post and time receive the most comments on average. Based on our analysis, to maximize the amount of comments a post receives, we'd recommend the post be categorized as ask post and created between 15:00 and 16:00(3:00 pm est - 4:00 pm est/8:00 pm WAT - 9:00 pm WAT).

However, it should be noted that the data set we analyzed excluded posts without any comments. Given that, it's more accurate to say that of the posts that received comments, ask posts received more comments on average and ask posts created between 15:00 and 16:00(3:00 pm est - 4:00 pm est/8:00 pm WAT - 9:00 pm WAT) received the most comments on average.

# Further Investigation on the dataset

Here, we will:
* Determine if show or ask posts receive more points on average.
* Determine if posts created at a certain time are more likely to receive more points.
* Compare your results to the average number of comments and points other posts receive.

In [11]:
# We proceed to determine if show or ask posts receive more point on the average

total_ask_points = 0
total_show_points = 0
for each_list in ask_posts:
    num_points = each_list[3]
    num_points = int(num_points)
    total_ask_points = total_ask_points + num_points
for each_list in show_posts:
    num_points = each_list[3]
    num_points = int(num_points)
    total_show_points = total_show_points + num_points
avg_ask_points = total_ask_points / len(ask_posts)
avg_show_points = total_show_points / len(show_posts)
print("Average Points for Ask Posts:", avg_ask_points)
print("Average Points for Show Posts:", avg_show_points)
    

Average Points for Ask Posts: 15.061926605504587
Average Points for Show Posts: 27.555077452667813


In [35]:
points_per_hour = []
for each_list in show_posts:
    points = each_list[3]
    hours = each_list[-1]
    points_per_hour.append([points, hours])
points_per_hour

# parse datetime using strptime method and format using strftime method to extract the hour only

num_points_per_hour = {}
hour = {}
for each_list in points_per_hour:
    date_time = each_list[1]
    points = int(each_list[0])
    date_time = dt.datetime.strptime(date_time, "%m/%d/%Y %H:%M")
    date_time = date_time.strftime("%H")
    if date_time in hour:
        hour[date_time] += 1
    else:
        hour[date_time] = 1
    if date_time in num_points_per_hour:
        num_points_per_hour[date_time] = (num_points_per_hour[date_time] + points) 
    else:
        num_points_per_hour[date_time] = points
average_points_per_hour = []
for each_hour in num_points_per_hour:
    ax = each_hour
    bx = num_points_per_hour[each_hour]
    cx = hour[each_hour]
    dx = bx / cx
    average_points_per_hour.append([each_hour, dx])
average_points_per_hour


    


[['01', 25.0],
 ['22', 40.34782608695652],
 ['00', 37.83870967741935],
 ['10', 18.916666666666668],
 ['05', 5.473684210526316],
 ['14', 25.430232558139537],
 ['19', 30.945454545454545],
 ['12', 41.68852459016394],
 ['21', 18.425531914893618],
 ['11', 33.63636363636363],
 ['07', 19.0],
 ['09', 18.433333333333334],
 ['06', 23.4375],
 ['04', 14.846153846153847],
 ['17', 27.107526881720432],
 ['23', 42.388888888888886],
 ['02', 11.333333333333334],
 ['13', 24.626262626262626],
 ['16', 28.322580645161292],
 ['03', 25.14814814814815],
 ['15', 28.564102564102566],
 ['20', 30.316666666666666],
 ['08', 15.264705882352942],
 ['18', 36.31147540983606]]

In [43]:
avg_points_hour = []
for each_list in average_points_per_hour:
    avg_points_hour.append([each_list[1], each_list[0]])
avg_points = sorted(avg_points_hour, reverse = True)
print("Top 5 hours that have the highest points for show HN post:")
avg_points[:5]

Top 5 hours that have the highest points for show HN post:


[[42.388888888888886, 23],
 [41.68852459016394, 12],
 [40.34782608695652, 22],
 [37.83870967741935, 0],
 [36.31147540983606, 18]]

In [63]:
a = avg_points[:20]
for each_list in a:
    points = each_list[0]
    hour = str(each_list[1])
    hour = dt.datetime.strptime(hour, "%H").strftime("%H:%M")
    template = "There are an average of {point:.2f} points per show HN post made at {time}"
    output = template.format(point = points, time = hour)
    print(output)
  

There are an average of 42.39 points per show HN post made at 23:00
There are an average of 41.69 points per show HN post made at 12:00
There are an average of 40.35 points per show HN post made at 22:00
There are an average of 37.84 points per show HN post made at 00:00
There are an average of 36.31 points per show HN post made at 18:00
There are an average of 33.64 points per show HN post made at 11:00
There are an average of 30.95 points per show HN post made at 19:00
There are an average of 30.32 points per show HN post made at 20:00
There are an average of 28.56 points per show HN post made at 15:00
There are an average of 28.32 points per show HN post made at 16:00
There are an average of 27.11 points per show HN post made at 17:00
There are an average of 25.43 points per show HN post made at 14:00
There are an average of 25.15 points per show HN post made at 03:00
There are an average of 25.00 points per show HN post made at 01:00
There are an average of 24.63 points per show HN

We further investigated  ask posts and show posts to determine which type of post and time receive the most points on average. Based on our analysis, although the highest number of points received for a show HN post was submitted by 23:00 US Eastern Time( 4:00 am West African Time), it seems that points associated with show posts do not correlate with the time it was submitted. From our result, the dispersion of the post hours was high: times were as dispersed as 00:00 to 23:00 but with some very strong central tendencies from the corresponding post points. It seems that the kind of post submitted determines the points received as against the post time. As such further investigation is recommended.
